In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet

# 1. Define the DocumentClassifier model architecture
class DocumentClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(DocumentClassifier, self).__init__()
        
        # EfficientNet backbone
        self.backbone = EfficientNet.from_pretrained('efficientnet-b0')
        
        # Global average pooling
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        # Classifier head
        num_features = self.backbone._fc.in_features
        self.classifier = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        # Extract features using the backbone
        x = self.backbone.extract_features(x)
        
        # Global average pooling
        x = self.global_pool(x)
        
        # Flatten the output tensor
        x = x.view(x.size(0), -1)
        
        # Classifier
        x = self.classifier(x)
        
        return x

# 2. Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate the model and load the state dict
model = DocumentClassifier(num_classes=3)
model.load_state_dict(torch.load('/mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/notebooks/document_classifier.pth', map_location=device))

# Move the model to the appropriate device
model.to(device)

# Set the model to evaluation mode
model.eval()

# 3. Preprocess the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the input size expected by EfficientNet
    transforms.ToTensor(),           # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
])

# Load your image
image_path = '/mnt/c/Users/Rahul/Desktop/aayush.jpg'
image = Image.open(image_path)

# Preprocess the image
image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to the device

# 4. Perform inference
with torch.no_grad():  # Disable gradient calculation for inference
    output = model(image)

# 5. Process the output
# The output is a probability distribution over the classes
predicted_probabilities = output.cpu().numpy()

# Get the predicted class
predicted_class = torch.argmax(output, dim=1).item()

print(f'Predicted class: {predicted_class}')
print(f'Class probabilities: {predicted_probabilities}')


Loaded pretrained weights for efficientnet-b0


/tmp/ipykernel_3547/2831277396.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/mnt/c/Users/Rahul/Desktop/Document-and-Record-Manageme

Predicted class: 0
Class probabilities: [[1.0000000e+00 9.6527951e-20 7.5662624e-18]]
